## Exercise-1: Ring of Fire

In this exercise, we will download the USGS earthquake catalogue for events larger than 6 from 1900 to early 2025. We will plot the events on a map of the globe to see the ring of fire and the concentration of earthquakes along the plate boundaries.

Note: the downloaded catalogue covers events up to early 2025.

First, we will import the required libraries.

In [ ]:
# 1) Install dependencies if missing
import sys, subprocess, importlib

def ensure(pkg, import_name=None):
    name = import_name or pkg
    try:
        importlib.import_module(name)
        print(f"[ok] {pkg} already installed")
    except Exception:
        print(f"[info] Installing {pkg} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
        importlib.import_module(name)
        print(f"[ok] {pkg} installed")


pkgs = ["os", "re", "requests", "zipfile", "geopandas",
        "matplotlib", "shapely", "cartopy", "obspy", 
        "folium", "osmnx", "numpy", "pandas"]

for pkg in pkgs:
    ensure(pkg)

In [ ]:
import os
import re
import requests
import zipfile
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import box    
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
from obspy.geodetics import gps2dist_azimuth
from obspy import read_events
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature


We will now define the parameters used to download the USGS catalogue. The two most important parameters are the start and end data and the maximum magnitude.

Please note that reducing the magnitude threshold could lead to extensive catalogues, as for each unit increase in magnitude, the number of earthquakes decreases by about a factor of 10, approximately.

In [ ]:
STARTTIME = UTCDateTime("1900-01-01")   
ENDTIME = UTCDateTime("2025-01-01")

# The catalogue should be for global events
CATALOG_CLIENT = Client("USGS")
MIN_MAG = 6.0  # minimum magnitude to query

# Request the catalogue using obspy and then save it to a file
CATALOG_CLIENT.get_events(starttime=STARTTIME, endtime=ENDTIME, minmagnitude=MIN_MAG,
                                filename = "usgs_eq_catalog_global.xml")

Now that we have downloaded the catalogue, we will read it first and check the number of events. Then we will remove events without depth or magnitude information.

In [ ]:
# The catalogue is saved as an xml file in the current directory
cat = read_events("usgs_eq_catalog_global.xml")

# Print number of events in the catalogue
print(f"Number of events in the catalogue: {len(cat)}")

# Remove events with missing depth or magnitude
cat = [event for event in cat if event.origins[0].depth is not 
       None and event.magnitudes[0].mag is not None]
print(f"Number of events with depth and magnitude information: {len(cat)}")

Now we will plot the earthquake catalogue centered at the location of the ring of fire to see the earthquakes' distribution at the plate boundaries. The earthquakes will be colored by depth to show the deep-seated earthquakes associated with the subduction zones specifically.

In [ ]:
# Create Cartopy map
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson(central_longitude=180))

# Add features to the map
ax.coastlines()
ax.stock_img()
ax.add_feature(cfeature.BORDERS, linestyle=':')
# ax.add_feature(cfeature.LAND, facecolor='lightgray')
# ax.add_feature(cfeature.OCEAN, facecolor='aqua')

# Plot earthquake events colored by depth and sized by magnitude

lons = [event.origins[0].longitude for event in cat]
lats = [event.origins[0].latitude for event in cat]
depths = [event.origins[0].depth for event in cat]  # in meters
depths = [float(depth)/1000.0 for depth in depths]  # convert to km    
mags = [event.magnitudes[0].mag for event in cat]
sc = ax.scatter(lons, lats, c=depths, s=[(mag**2) for mag in mags], 
                cmap='autumn', alpha=0.7, transform=ccrs.PlateCarree())

plt.colorbar(sc, label='Depth (km)', orientation='horizontal', pad=0.02, aspect=30, shrink=0.5)
plt.xlabel("Longitude")
plt.ylabel("Latitude")

plt.title("Global Earthquake Catalogue (M>=6.5) Centered at (0, 180)")
plt.show()

Now we will explore the relation between the earthqukes magnitude and depths and their frequency of occurence.

In [ ]:
# %% Make a histgram of the earthquake magnitudes in the catalogue
# and their frequencies. Use magnitude bin of 0.1
import numpy as np
mags = [event.magnitudes[0].mag for event in cat]
plt.figure(figsize=(10, 6))
plt.hist(mags, bins=np.arange(MIN_MAG, max(mags)+0.1, 0.1), edgecolor='black', alpha=0.7)
plt.xlabel("Magnitude")     
plt.ylabel("Frequency")
plt.title("Histogram of Earthquake Magnitudes (M>=6.0) Frequencies")
plt.grid()
plt.show()

In [ ]:
# %% Make a histogram of the earthquake depths in the catalogue
# and their frequencies. Use depth bin of 10 km
depths = [event.origins[0].depth for event in cat]  # in       
depths = [float(depth)/1000.0 for depth in depths]  # convert to km
plt.figure(figsize=(10, 6))
plt.hist(depths, bins=np.arange(0, max(depths)+10, 10), edgecolor='black', alpha=0.7)
plt.xlabel("Depth (km)")
plt.ylabel("Frequency")
plt.title("Histogram of Earthquake Depths (M>=6.0)")
plt.grid()
plt.show()

Do you have any general observations about the histograms? Can you link them to general geological or tectonic common knwoledge?

# Extra tasks
Try to change the search criteria to match your country. Search for all earthquakes larger than 4 or 3.5 and plot the catalogue.
* Is your country near the Ring of Fire?
* What is the strongest earthquake ever observed there?
* Is it related to a specific geologic or tectonic feature?
* Do you know the name of the fault associated with the strongest earthquake?
* What is the relation between the earthquake magnitudes, depths, and their frequency of occurrence?
* Can you link these relations to known geological or tectonic terrains?